In [1]:
from model.classification_model import ClassifierModel # class with classifiers
from model.reduce_model import ReduceModel # class with autoencoder

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from plot_settings import * # plot configuration 
import joblib

In [3]:
scaler = joblib.load("./best/scaler.pkl")
reduce = joblib.load("./best/reduce_model.pkl")
clf = joblib.load("./best/c_model.pkl")
cifs = pd.read_csv("./main_datasets/test_cifs.csv", index_col=0)

In [4]:
cifs_reduced = reduce.transform(cifs)

In [6]:
df = pd.DataFrame(cifs_reduced, index=cifs.index)

In [7]:
df

,0,1,2,3,4,5,6,7,8,9
1-final,3.284673,-1.422195,-2.102811,-4.860344,-1.080255,-4.728310,-1.092881,0.019246,-1.420921,1.190130
2-final,3.286075,-1.421376,-2.104950,-4.858925,-1.080578,-4.731123,-1.093828,0.016528,-1.417691,1.189848
2080270,0.567122,0.085580,-2.411250,-4.089851,-0.203309,-2.979827,0.860138,3.009019,-2.821081,2.846363
2080334,0.492588,0.093742,-2.356460,-3.997657,-0.172882,-2.969380,0.694580,3.049262,-2.787228,2.852643
CSV157 initial,3.251403,-2.384271,-3.966932,-4.962783,-3.109287,-4.289739,0.645156,-1.248142,-2.628460,1.635036
CSV158 initial,3.412864,-2.354106,-3.946173,-5.075735,-3.140628,-4.274045,0.766914,-1.158762,-2.728949,1.622409


In [25]:
predicted_proba = pd.DataFrame(columns=clf.models.keys())
def verdict(proba):
    if proba > 0.8:
        return "guest mol."
    if proba < 0.2:
        return "T-P"
    return "No prediction"
for m in clf.models:
    print(m)
    proba = clf.models[m].predict_proba(df)[:,1]
    d = pd.DataFrame(columns=["Predicted probability for guest mol. s.t.", "verdict"])
    for i, cif_name in enumerate(cifs.index):
        d.loc[cif_name] = [proba[i], verdict(proba[i])]
    print(d)

CatBoost
                Predicted probability for guest mol. s.t.        verdict
1-final                                          0.499884  No prediction
2-final                                          0.499884  No prediction
2080270                                          0.500172  No prediction
2080334                                          0.500117  No prediction
CSV157 initial                                   0.500367  No prediction
CSV158 initial                                   0.500244  No prediction
RF
                Predicted probability for guest mol. s.t.        verdict
1-final                                          0.510638  No prediction
2-final                                          0.510638  No prediction
2080270                                          0.468085  No prediction
2080334                                          0.478723  No prediction
CSV157 initial                                   0.531915  No prediction
CSV158 initial                         

In [10]:
clf.models

{'CatBoost': <catboost.core.CatBoostClassifier at 0x1c676bf2c50>,
 'RF': RandomForestClassifier(max_depth=14, n_estimators=94, random_state=42),
 'AdaBoost': AdaBoostClf(learning_rate=0.0004257864761068253, max_depth=11, n_estimators=55,
             random_state=42),
 'Knn': KNeighborsClassifier(n_neighbors=3, weights='distance')}